In [ ]:
!pip install -q transformers datasets evaluate scikit-learn accelerate
from datasets import load_dataset

# Load the SCOTUS split from LexGLUE
dataset = load_dataset("lex_glue", "scotus")

# Check dataset structure
print(dataset)
print(dataset["train"][0])

# Get label names directly from dataset
label_names = dataset["train"].features["label"].names
num_labels = len(label_names)
print("Number of labels:", num_labels)
print("Labels:", label_names)


from transformers import AutoTokenizer

# Use Legal-BERT for legal domain texts
model_name = "nlpaueb/legal-bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=512)

tokenized = dataset.map(tokenize, batched=True)


from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels
)

from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
import transformers

print("Transformers version:", transformers.__version__)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "macro_f1": f1_score(labels, preds, average="macro")
    }

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=8,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
    fp16=True,
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()


metrics = trainer.evaluate(tokenized["test"])
print("\nFinal Test Metrics:", metrics)


from sklearn.metrics import classification_report

preds_output = trainer.predict(tokenized["test"])
y_true = preds_output.label_ids
y_pred = np.argmax(preds_output.predictions, axis=1)

print("\nDetailed Classification Report:\n")
print(classification_report(y_true, y_pred, target_names=label_names))


save_directory = "./legalbert_scotus_model"
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)
print(f"\nModel and tokenizer saved successfully at: {save_directory}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.3 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

scotus/train-00000-of-00001.parquet:   0%|          | 0.00/94.4M [00:00<?, ?B/s]

scotus/test-00000-of-00001.parquet:   0%|          | 0.00/40.0M [00:00<?, ?B/s]

...

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jp-developer1204 (jp-developer1204-navrachana-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
1,1.259600,0.935524,0.703571,0.514794
2,0.703600,0.873533,0.749286,0.638271
3,0.510900,0.891372,0.767857,0.688699
4,0.268800,1.112505,0.754286,0.663707
5,0.153500,1.138800,0.772143,0.697705
6,0.130000,1.289748,0.776429,0.694649
7,0.083900,1.377005,0.773571,0.695155
8,0.037400,1.386775,0.766429,0.690086



Final Test Metrics: {'eval_loss': 1.3741693496704102, 'eval_accuracy': 0.7371428571428571, 'eval_macro_f1': 0.6171304008931633, 'eval_runtime': 10.5724, 'eval_samples_per_second': 132.421, 'eval_steps_per_second': 16.553, 'epoch': 8.0}

Detailed Classification Report:

              precision    recall  f1-score   support

           1       0.82      0.93      0.87       372
           2       0.77      0.75      0.76       222
           3       0.80      0.93      0.86        88
           4       0.60      0.57      0.59        51
           5       0.50      0.39      0.44        28
           6       0.76      0.76      0.76        17
           7       0.75      0.75      0.75        24
           8       0.65      0.79      0.71       260
           9       0.66      0.55      0.60       200
          10       0.46      0.14      0.22        83
          11       0.75      0.40      0.52        15
          12       0.94      0.92      0.93        37
          13       0.00   

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Model and tokenizer saved successfully at: ./legalbert_scotus_model
